# The Second Script
### In this script, the corpus will be streamed into a gensim Word2Vec model and the model embeddings will be trained. 

Here are the different things to play/tune around here:

1. Custom corpus object and final processing step. Wondering why there's a weird corpus object? Hint: it's **memory**.  Check [here](https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html#corpus-streaming-tutorial)! 
2. All the hyperparameters for the model! Dimensionality, minimum word count, number of worker threads, context window, and downsampling can all be changed here. Number of epochs isn't a huge deal, check out this [Stack Overflow answer](https://stackoverflow.com/a/46857922). 
3. Choice of colocation of common phrases - gensim.models.phrases.Phrases is currently used. For brevity sake, the Phrases model is not trained repeatedly, TODO - Phrases should be trained on non-replaced corpus. 

**Note**: on a 2018 MacBook Pro, the training took roughly an hour. Go and cook a nice meal, read a book, get some exercise in and let it run! 

In [1]:
import gensim
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.phrases import Phrases, Phraser
import logging
import multiprocessing

In [3]:
# Making a custom corpus object to stream in the corpus line by line
# so the whole corpus doesn't need to be in memory at once

class HansardCorpus(object):
    corpus_paths = {('abort', n):f'../corpora/corpus_abort_{n}.txt' for n in range(1,7)}
    corpus_paths[('no_stopwords',1)] = '../corpora/corpus.txt'
    corpus_paths[('rolling',1)] = '../corpora/corpus_abort_3_rolling.txt'
    def __init__(self, replacement='abort', group=1):
        self.corpus_path = self.corpus_paths[(replacement, group)]
    
    def __iter__(self):
        for line in open(self.corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.split()    
            

In [11]:
group = 3 # Choice of corpus groupings, 1-6 available
num_features = 300 # Word vector dimensionality
min_word_count = [10,15,18,30,30,30] # Minimum word counts, for each group
num_workers = multiprocessing.cpu_count() # Number of threads to run in parallel
context = 6 # Context window size
downsampling = 1e-3 # Downsample setting for frequent words

sentences = HansardCorpus('abort',3)
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)

The below three lines are for training the bigram, it won't need to be run every time. 

In [12]:
# bigram_transformer = Phrases(sentences, progress_per=200000)
# bigram = Phraser(bigram_transformer)
# bigram.save("../data/models/bigram.pkl")

In [13]:
sentences.corpus_path

'../corpora/corpus_abort_3.txt'

In [15]:
# Loading bigram and training the model
rep = 1 # replicate number for multiple models 
bigram = Phraser.load("../models/bigram.pkl")
model = word2vec.Word2Vec(bigram[sentences], workers=num_workers, 
                          size=num_features, min_count=min_word_count[group-1], 
                          window=context, sample=downsampling)

model.init_sims(replace=True) # initialize L2-normalized vectors for comparing similarity 
model_name = '../models/lipadmodel_{}_{}'.format(group, rep)
model.save(model_name)

2020-06-13 12:48:54,150 : loading Phraser object from ../models/bigram.pkl
2020-06-13 12:48:54,458 : loaded ../models/bigram.pkl
2020-06-13 12:48:54,501 : collecting all words and their counts
2020-06-13 12:48:54,502 : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-13 12:48:54,742 : PROGRESS: at sentence #10000, processed 109244 words, keeping 13405 word types
2020-06-13 12:48:54,974 : PROGRESS: at sentence #20000, processed 220426 words, keeping 20908 word types
2020-06-13 12:48:55,227 : PROGRESS: at sentence #30000, processed 329675 words, keeping 25852 word types
2020-06-13 12:48:55,435 : PROGRESS: at sentence #40000, processed 421952 words, keeping 29903 word types
2020-06-13 12:48:55,673 : PROGRESS: at sentence #50000, processed 523504 words, keeping 33709 word types
2020-06-13 12:48:55,917 : PROGRESS: at sentence #60000, processed 623697 words, keeping 37044 word types
2020-06-13 12:48:56,163 : PROGRESS: at sentence #70000, processed 723274 words, keepi

2020-06-13 12:49:12,994 : PROGRESS: at sentence #750000, processed 7438305 words, keeping 160752 word types
2020-06-13 12:49:13,243 : PROGRESS: at sentence #760000, processed 7538743 words, keeping 162300 word types
2020-06-13 12:49:13,482 : PROGRESS: at sentence #770000, processed 7636294 words, keeping 163911 word types
2020-06-13 12:49:13,730 : PROGRESS: at sentence #780000, processed 7737820 words, keeping 165361 word types
2020-06-13 12:49:13,980 : PROGRESS: at sentence #790000, processed 7840765 words, keeping 166577 word types
2020-06-13 12:49:14,245 : PROGRESS: at sentence #800000, processed 7948631 words, keeping 168082 word types
2020-06-13 12:49:14,496 : PROGRESS: at sentence #810000, processed 8052219 words, keeping 169168 word types
2020-06-13 12:49:14,751 : PROGRESS: at sentence #820000, processed 8160081 words, keeping 170473 word types
2020-06-13 12:49:15,003 : PROGRESS: at sentence #830000, processed 8263442 words, keeping 171742 word types
2020-06-13 12:49:15,231 : PR

2020-06-13 12:49:31,757 : PROGRESS: at sentence #1500000, processed 15068651 words, keeping 258509 word types
2020-06-13 12:49:32,022 : PROGRESS: at sentence #1510000, processed 15175041 words, keeping 259669 word types
2020-06-13 12:49:32,274 : PROGRESS: at sentence #1520000, processed 15278022 words, keeping 260716 word types
2020-06-13 12:49:32,523 : PROGRESS: at sentence #1530000, processed 15379398 words, keeping 262066 word types
2020-06-13 12:49:32,739 : PROGRESS: at sentence #1540000, processed 15471875 words, keeping 263034 word types
2020-06-13 12:49:32,988 : PROGRESS: at sentence #1550000, processed 15573023 words, keeping 264105 word types
2020-06-13 12:49:33,237 : PROGRESS: at sentence #1560000, processed 15674636 words, keeping 265148 word types
2020-06-13 12:49:33,509 : PROGRESS: at sentence #1570000, processed 15782170 words, keeping 266415 word types
2020-06-13 12:49:33,782 : PROGRESS: at sentence #1580000, processed 15894928 words, keeping 267598 word types
2020-06-13

2020-06-13 12:49:49,112 : PROGRESS: at sentence #2250000, processed 22730592 words, keeping 336896 word types
2020-06-13 12:49:49,365 : PROGRESS: at sentence #2260000, processed 22838579 words, keeping 338134 word types
2020-06-13 12:49:49,600 : PROGRESS: at sentence #2270000, processed 22942339 words, keeping 339105 word types
2020-06-13 12:49:49,819 : PROGRESS: at sentence #2280000, processed 23039287 words, keeping 340036 word types
2020-06-13 12:49:50,053 : PROGRESS: at sentence #2290000, processed 23139050 words, keeping 340776 word types
2020-06-13 12:49:50,316 : PROGRESS: at sentence #2300000, processed 23252423 words, keeping 341699 word types
2020-06-13 12:49:50,572 : PROGRESS: at sentence #2310000, processed 23362359 words, keeping 342867 word types
2020-06-13 12:49:50,791 : PROGRESS: at sentence #2320000, processed 23461156 words, keeping 344018 word types
2020-06-13 12:49:51,030 : PROGRESS: at sentence #2330000, processed 23563715 words, keeping 345189 word types
2020-06-13

2020-06-13 12:50:06,948 : PROGRESS: at sentence #3000000, processed 30182272 words, keeping 413746 word types
2020-06-13 12:50:07,177 : PROGRESS: at sentence #3010000, processed 30282537 words, keeping 415051 word types
2020-06-13 12:50:07,403 : PROGRESS: at sentence #3020000, processed 30377176 words, keeping 415739 word types
2020-06-13 12:50:07,629 : PROGRESS: at sentence #3030000, processed 30475410 words, keeping 416586 word types
2020-06-13 12:50:07,848 : PROGRESS: at sentence #3040000, processed 30571730 words, keeping 417333 word types
2020-06-13 12:50:08,068 : PROGRESS: at sentence #3050000, processed 30665302 words, keeping 418442 word types
2020-06-13 12:50:08,303 : PROGRESS: at sentence #3060000, processed 30767061 words, keeping 419198 word types
2020-06-13 12:50:08,536 : PROGRESS: at sentence #3070000, processed 30869516 words, keeping 419847 word types
2020-06-13 12:50:08,772 : PROGRESS: at sentence #3080000, processed 30976010 words, keeping 420713 word types
2020-06-13

...

2020-06-13 13:54:04,094 : EPOCH 5 - PROGRESS: at 54.81% examples, 313473 words/s, in_qsize 14, out_qsize 1
2020-06-13 13:54:05,115 : EPOCH 5 - PROGRESS: at 54.93% examples, 313405 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:54:06,141 : EPOCH 5 - PROGRESS: at 55.06% examples, 313423 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:54:07,163 : EPOCH 5 - PROGRESS: at 55.22% examples, 313557 words/s, in_qsize 10, out_qsize 0
2020-06-13 13:54:08,227 : EPOCH 5 - PROGRESS: at 55.36% examples, 313546 words/s, in_qsize 13, out_qsize 0
2020-06-13 13:54:09,237 : EPOCH 5 - PROGRESS: at 55.47% examples, 313487 words/s, in_qsize 16, out_qsize 0
2020-06-13 13:54:10,262 : EPOCH 5 - PROGRESS: at 55.62% examples, 313570 words/s, in_qsize 11, out_qsize 0
2020-06-13 13:54:11,283 : EPOCH 5 - PROGRESS: at 55.71% examples, 313368 words/s, in_qsize 14, out_qsize 0
2020-06-13 13:54:12,401 : EPOCH 5 - PROGRESS: at 55.85% examples, 313320 words/s, in_qsize 16, out_qsize 0
2020-06-13 13:54:13,507 : EPOCH 5 - P

2020-06-13 13:55:24,775 : EPOCH 5 - PROGRESS: at 66.07% examples, 312201 words/s, in_qsize 9, out_qsize 3
2020-06-13 13:55:25,905 : EPOCH 5 - PROGRESS: at 66.23% examples, 312156 words/s, in_qsize 14, out_qsize 1
2020-06-13 13:55:27,031 : EPOCH 5 - PROGRESS: at 66.39% examples, 312150 words/s, in_qsize 15, out_qsize 1
2020-06-13 13:55:28,324 : EPOCH 5 - PROGRESS: at 66.55% examples, 312039 words/s, in_qsize 14, out_qsize 1
2020-06-13 13:55:29,348 : EPOCH 5 - PROGRESS: at 66.71% examples, 312078 words/s, in_qsize 10, out_qsize 3
2020-06-13 13:55:30,369 : EPOCH 5 - PROGRESS: at 66.86% examples, 312040 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:55:31,444 : EPOCH 5 - PROGRESS: at 67.01% examples, 312012 words/s, in_qsize 16, out_qsize 0
2020-06-13 13:55:32,491 : EPOCH 5 - PROGRESS: at 67.17% examples, 311998 words/s, in_qsize 12, out_qsize 3
2020-06-13 13:55:33,683 : EPOCH 5 - PROGRESS: at 67.35% examples, 311972 words/s, in_qsize 11, out_qsize 5
2020-06-13 13:55:34,683 : EPOCH 5 - PR

2020-06-13 13:56:46,712 : EPOCH 5 - PROGRESS: at 77.61% examples, 310641 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:56:47,738 : EPOCH 5 - PROGRESS: at 77.77% examples, 310632 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:56:48,841 : EPOCH 5 - PROGRESS: at 77.93% examples, 310616 words/s, in_qsize 16, out_qsize 0
2020-06-13 13:56:49,898 : EPOCH 5 - PROGRESS: at 78.09% examples, 310607 words/s, in_qsize 9, out_qsize 6
2020-06-13 13:56:50,919 : EPOCH 5 - PROGRESS: at 78.23% examples, 310535 words/s, in_qsize 13, out_qsize 2
2020-06-13 13:56:52,042 : EPOCH 5 - PROGRESS: at 78.39% examples, 310539 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:56:53,118 : EPOCH 5 - PROGRESS: at 78.55% examples, 310536 words/s, in_qsize 14, out_qsize 1
2020-06-13 13:56:54,136 : EPOCH 5 - PROGRESS: at 78.71% examples, 310530 words/s, in_qsize 16, out_qsize 0
2020-06-13 13:56:55,177 : EPOCH 5 - PROGRESS: at 78.87% examples, 310577 words/s, in_qsize 13, out_qsize 0
2020-06-13 13:56:56,367 : EPOCH 5 - PR

2020-06-13 13:58:07,430 : EPOCH 5 - PROGRESS: at 89.09% examples, 309286 words/s, in_qsize 12, out_qsize 3
2020-06-13 13:58:08,439 : EPOCH 5 - PROGRESS: at 89.24% examples, 309273 words/s, in_qsize 14, out_qsize 1
2020-06-13 13:58:09,553 : EPOCH 5 - PROGRESS: at 89.40% examples, 309283 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:58:10,614 : EPOCH 5 - PROGRESS: at 89.55% examples, 309273 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:58:11,704 : EPOCH 5 - PROGRESS: at 89.71% examples, 309264 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:58:12,739 : EPOCH 5 - PROGRESS: at 89.84% examples, 309211 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:58:13,754 : EPOCH 5 - PROGRESS: at 89.99% examples, 309223 words/s, in_qsize 15, out_qsize 0
2020-06-13 13:58:14,914 : EPOCH 5 - PROGRESS: at 90.15% examples, 309197 words/s, in_qsize 14, out_qsize 1
2020-06-13 13:58:15,938 : EPOCH 5 - PROGRESS: at 90.31% examples, 309247 words/s, in_qsize 12, out_qsize 0
2020-06-13 13:58:17,055 : EPOCH 5 - P

2020-06-13 13:59:26,798 : worker thread finished; awaiting finish of 4 more threads
2020-06-13 13:59:26,800 : worker thread finished; awaiting finish of 3 more threads
2020-06-13 13:59:26,811 : worker thread finished; awaiting finish of 2 more threads
2020-06-13 13:59:26,812 : worker thread finished; awaiting finish of 1 more threads
2020-06-13 13:59:26,814 : worker thread finished; awaiting finish of 0 more threads
2020-06-13 13:59:26,814 : EPOCH - 5 : training on 240135144 raw words (226902401 effective words) took 736.3s, 308167 effective words/s
2020-06-13 13:59:26,815 : training on a 1200675720 raw words (1134525389 effective words) took 3621.4s, 313285 effective words/s
2020-06-13 13:59:27,163 : precomputing L2-norms of word weight vectors
2020-06-13 13:59:27,313 : saving Word2Vec object under ../models/lipadmodel_3_1, separately None
2020-06-13 13:59:27,314 : storing np array 'vectors' to ../models/lipadmodel_3_1.wv.vectors.npy
2020-06-13 13:59:27,771 : not storing attribute vec